In [49]:

%load_ext autoreload
%autoreload 2
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/6/7 15:17
# @Author  : Wang Yujia
# @File    : mlp.ipynb
# @Description : 搭一个基本的mlp，用于测试思路
# @TODO: 画图细化一下training的流程：怎么用target data/ NN的规模/ batch size等


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 0. what for
1. 搭一个基本的mlp

# 1. Preparations
## 1.1 global settings

In [29]:
import pandas as pd

# nums of Gaussian kernels
N_gaussians = 3

# dataset划分
batch_size = 1
train_percentage = 0.8
vali_percentage = 0.2

# train and optim.
learning_rate = 0.01
total_train_step = 0
total_test_step = 0
epoch = 5

# import
from BasicInfo.common_settings import *
# from torchsummary import summary
from torch.utils.data import DataLoader
import pandas as pd
import torch.utils.data
from mydataset import *
# from tensorboardX import SummaryWriter

ModuleNotFoundError: No module named 'torch'

## 1.2 the data path
1. 注意target data和PT的data一样：drop掉duration>T的samples

In [28]:
# input data
GT_1_data_path = "../data/info_asymm/results/asc_symmetry/GT_asc_symmetry_P2_K=300.csv"
GT_2_data_path = "../data/SA_PT/results/PT_oneforall_P_K=300.csv"

# target data
target_data_path = "../data/targets/target_data_NP.csv"
target_output_head= "../data/targets/target_data_NP_"
target_output_tail= ".csv"

# data keys (for target)
data_key_path = "../data/targets/target_datakey.csv"

# 2. Dataset read and split
1. 80%是training data？
2. 划分是按照target_data的data_key来划分的


In [31]:
data_1 = pd.read_csv(GT_1_data_path)
data_i = data_1[(data_1['bidincrement'] == 0.15) & (data_1['bidfee'] == 0.75) &(data_1['retail'] == 169.99)]
data_i.shape

(2, 303)

In [52]:
# GT_1_data = mydataset([GT_1_data_path,GT_2_data_path],target_data_path,data_key_path)
data_key = pd.read_csv(data_key_path)
data_key.head()

,product_id,bidincrement,bidfee,retail
0,10009881,0.15,0.75,169.99
1,10006115,0.15,0.75,499.99
2,10007148,0.15,0.75,299.99
3,10007263,0.15,0.75,89.99
4,10010575,0.15,0.75,59.99


## 2.1 Dataloader

In [53]:
data = myDataset([GT_1_data_path, GT_2_data_path, target_output_head, data_key_path])

In [54]:
train_size = int(train_percentage * data.__len__())
vali_size = data.__len__() - train_size
train_data, vali_data= torch.utils.data.random_split(data, [train_size, vali_size])

In [55]:
data.__getitem__(0)

D:\Desktop\PROJ\PAProj\MLP\mydataset.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # select target data with the key


TypeError: Cannot change data-type for object array.

In [ ]:
# 1.2 加载dataloader
train_iter = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=False)
vali_iter = DataLoader(vali_data, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=False)

In [3]:
# data keys (for target)
import pandas as pd
data_key_path = "../data/targets/target_datakey.csv"
data_key_all = pd.read_csv(data_key_path)


In [9]:
data_key_all = data_key_all.copy()
data_key_all.loc[:,'product_id'] = data_key_all['product_id'].values.astype("int64")

In [26]:
data_key_i = data_key_all.iloc[0,:]
data_key_i = data_key_i.copy()
data_key_i[0] = data_key_i[0].astype("int64")
print(int(data_key_i[0]))
print(type(data_key_i[0]))
print(data_key_i[0] == 10009881)

10009881
<class 'numpy.float64'>
True
